In [ ]:
!pip install -q duckdb dbt-duckdb ducklake-delta-exporter  --upgrade
import sys
sys.exit(0)

In [ ]:
import os
os.environ['DBT_SCHEMA']              = 'aemo'  
os.environ['ROOT_PATH']               = 'abfss://dbt@onelake.dfs.fabric.microsoft.com/data.Lakehouse'
os.environ['download_limit']          = '10'

In [ ]:
!git clone https://github.com/djouallah/dbt.git /tmp/dbt
!cd /tmp/dbt && dbt run && dbt test

In [ ]:
from ducklake_delta_exporter import generate_latest_delta_log
generate_latest_delta_log('/tmp/metadata.db')

In [ ]:
import duckdb
duckdb.sql(f"""select max(cutoff), count(*) from delta_scan
('{os.environ['ROOT_PATH']}/Tables/{os.environ['DBT_SCHEMA']}/fct_summary') 
 """).show()